In [1]:
import pandas as pd
import numpy as np

import re
from itertools import chain
from collections import Counter

from google.cloud import bigquery
from google.oauth2 import service_account
import gspread
from gspread_dataframe import set_with_dataframe
from googleapiclient.discovery import build
import io
from googleapiclient.http import MediaIoBaseDownload

from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

# from fuzzywuzzy import fuzz  # fuzz 모듈 임포트
# from fuzzywuzzy import process

import requests
import warnings
from requests.packages.urllib3.exceptions import InsecureRequestWarning

# InsecureRequestWarning 경고 무시
warnings.simplefilter('ignore', InsecureRequestWarning)

In [2]:
KEY_PATH = ".config/"
servicekey_path = KEY_PATH + "serviceKey.json" ## 빅쿼리 외 다른 API 활용 위해
bigquerykey_path = KEY_PATH + "mido-project-426906-31b49963ac97.json"
sheetskey_path = KEY_PATH + "mido-project-426906-41a4b6d0e3db.json"
midopluskey_path = KEY_PATH + "midoplus.json"

warnings.filterwarnings("ignore")


In [3]:
# BigQuery 클라이언트 생성 함수
def create_bigquery_client(key_path):
    credentials = service_account.Credentials.from_service_account_file(key_path)
    client = bigquery.Client(credentials=credentials, project=credentials.project_id)
    return client

In [4]:
def save_dataframe_to_bigquery(df, dataset_id, table_id, key_path):
    # BigQuery 클라이언트 객체 생성
    client = create_bigquery_client(key_path)

    # 테이블 레퍼런스 생성
    table_ref = client.dataset(dataset_id).table(table_id)

    # 데이터프레임을 BigQuery 테이블에 적재
    job_config = bigquery.LoadJobConfig()
    job_config.write_disposition = "WRITE_TRUNCATE"  # 기존 테이블 내용 삭제 후 삽입

    job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
    job.result()  # 작업 완료 대기

    print(f"Data inserted into table {table_id} successfully.")

In [5]:
def get_dataframe_from_bigquery(dataset_id, table_id, key_path):
    # BigQuery 클라이언트 생성
    client = create_bigquery_client(key_path)

    # 테이블 레퍼런스 생성
    table_ref = client.dataset(dataset_id).table(table_id)

    # 테이블 데이터를 DataFrame으로 변환
    df = client.list_rows(table_ref).to_dataframe()

    return df

In [6]:
def get_dataframe_from_bigquery_budget(dataset_id, table_id, key_path, partition_date):
    # BigQuery 클라이언트 생성
    client = create_bigquery_client(key_path)

    # 쿼리 작성
    query = f"""
    SELECT *
    FROM `{dataset_id}.{table_id}`
    WHERE collection_Date = '{partition_date}'
    """

    # 테이블 데이터를 DataFrame으로 변환
    df = client.query(query).to_dataframe()

    return df

### 구글 스프레드 시트

In [7]:
# Google Sheets 클라이언트 생성
sheets_scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/spreadsheets",
                "https://www.googleapis.com/auth/drive.file", "https://www.googleapis.com/auth/drive"]

sheets_creds = service_account.Credentials.from_service_account_file(midopluskey_path, scopes=sheets_scope)
gc = gspread.authorize(sheets_creds)

In [8]:
# 오늘 날짜
today = datetime.today()#.strftime('%Y%m%d')

# 어제 날짜 계산
ytday = datetime.today() - timedelta(days=1)

# 어제 날짜 계산
ytday2 = datetime.today() - timedelta(days=2)

# 한달 날짜 계산
month = datetime.today() - relativedelta(months=1)

# 만약 어제, 오늘이 토요일(5) 또는 일요일(6)이라면, 그 전주 금요일로 변경
if today.weekday() == 5:  # 토요일
    today -= timedelta(days=1)
elif today.weekday() == 6:  # 일요일
    today -= timedelta(days=2)
if ytday.weekday() == 5:  # 토요일
    ytday -= timedelta(days=1)
elif ytday.weekday() == 6:  # 일요일
    ytday -= timedelta(days=2)
if ytday2.weekday() == 5:  # 토요일
    ytday2 -= timedelta(days=1)
elif ytday2.weekday() == 6:  # 일요일
    ytday2 -= timedelta(days=2)
if month.weekday() == 5:  # 토요일
    month -= timedelta(days=1)
elif month.weekday() == 6:  # 일요일
    month -= timedelta(days=2)

# 'YYYYMMDD' 형식으로 변환
ytday2 = ytday2.strftime('%Y%m%d')
ytday = ytday.strftime('%Y%m%d')
today = today.strftime('%Y%m%d')
month = month.strftime('%Y%m%d')

#### 법정동코드

In [9]:
dist_code = pd.read_csv('C:\py_src\midoproject\data/법정동코드 전체자료.csv',encoding='cp949')

In [10]:
dist_code_list = []
for i in range(dist_code['법정동명'].str.split().apply(len).max()):
    dist_code_temp = dist_code[dist_code['법정동명'].str.split().str[i].str[-1].isin(['읍','면','동','리','가','로']).dropna()]
    dist_code_list.append([dist_code_temp['법정동명'].str.split().str[i].dropna().unique()])

emd_code_list = list(chain(*chain(*dist_code_list)))

#### 종합쇼핑몰 납품상세내역

In [11]:
# all_shop_df = get_dataframe_from_bigquery('g2b', 'shop_detail_df_all', bigquerykey_path) ## 개인계정
all_shop_df = get_dataframe_from_bigquery('DATA_WAREHOUSE', 'g2b_data', midopluskey_path) ## 회사계정

In [12]:
years = today[:-4]
all_shop_df = all_shop_df[all_shop_df['납품요구접수일자'].str.split('-').str[0]==years].reset_index(drop=True)
all_shop_df_fin = all_shop_df[['납품요구접수일자', '수요기관명', '납품요구건명', '업체명', '단가', '단위', '수량', '금액', '수요기관코드', '수요기관구분', '수요기관지역명','납품요구지청명']]
all_shop_df_fin = all_shop_df_fin.sort_values(['납품요구접수일자'],ascending=False).reset_index(drop=True)

In [13]:
# 특수문자, 숫자, 영어 제거 함수 (연속된 공백을 단일 공백으로 변환 포함)
def clean_text(text):
    text = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣\s]', ' ', text)
    text = re.sub('\s+', ' ', text).strip()  # 연속된 공백을 단일 공백으로 변환
    return text

In [14]:
# 사업명 전처리
all_shop_df_fin['납품요구건명_re'] = all_shop_df_fin['납품요구건명'].apply(clean_text)
# all_shop_df_fin['납품요구건명_re'] = all_shop_df_fin['납품요구건명_re'].str.replace('초등학교','').str.replace('중학교','').str.replace('고등학교','')
all_shop_df_fin['납품요구건명_re'] = all_shop_df_fin['납품요구건명_re'].str.strip()

all_shop_df_fin['납품요구건명_re'] = all_shop_df_fin['납품요구건명_re'].str.split().apply(lambda x: [kw for kw in x if len(kw) > 1]) ## 글자수 1 제외
all_shop_df_fin['납품요구건명_re'] = all_shop_df_fin['납품요구건명_re'].apply((' ').join)

In [15]:
# 수요기관 필터링
dist_nm = pd.Series(all_shop_df_fin[all_shop_df_fin['수요기관지역명'].str.split(' ').str[1].notnull()]['수요기관지역명'].str.split(' ').str[1].unique())
dist_nm = np.where(dist_nm.apply(len)<=2,dist_nm,dist_nm.str[:-1])

#### 지자체 세부사업별 예산서

In [16]:
# # 스프레드시트 ID (URL에서 확인 가능)
# business_sheet_id = '166xdkZYI-SDNwdEiI6-Kt-p1wsTwFTAYAVfuCiFly0E' ## 미도플러스 사업현황

# # 기존 스프레드시트 및 워크시트 열기
# WORKSHEET_NAME = '지자체'
# spreadsheet_business = gc.open_by_key(business_sheet_id)
# worksheet_business = spreadsheet_business.worksheet(WORKSHEET_NAME)

# # 구글 시트에서 데이터 읽기
# budget_sheet = worksheet_business.get_all_records()
# budget_sheet_df = pd.DataFrame(budget_sheet)

In [17]:
# # 빅쿼리에서 불러오기(개인계정)
# budget_df_today = get_dataframe_from_bigquery('budget','budget_df_0' + today,bigquerykey_path)
# # budget_df_ytday = get_dataframe_from_bigquery('budget','budget_df_0' + ytday,bigquerykey_path)

In [18]:
# 빅쿼리에서 불러오기(회사계정)
part_date_today = today[:4] + '-' + today[4:6] + '-' + today[-2:]
# part_date_today = '2024-09-05'
budget_df_today = get_dataframe_from_bigquery_budget('DATA_WAREHOUSE','budget_data', midopluskey_path, part_date_today)
budget_df_today = budget_df_today.drop('collection_Date',axis=1)

In [19]:
# budget_df_today = budget_df_today[budget_df_today['세부사업명'].isin(budget_sheet_df['세부사업명'].unique())].reset_index(drop=True)

In [20]:
# 필요키워드1
need_kwd1 = '학교|초등학교|중학교|고등학교|대학교|다목적|운동장|종합운동장|공원|체육|체육공원|체육시설|체육센터|스포츠|스포츠센터|놀이시설|연병장|축구장|족구장|야구장|풋살장|배드민턴장|테니스장|게이트볼장|하키장|에어돔'

# 필요키워드2
need_kwd2 = '개선|교체|보수|공사|구매|구입|설치|정비|조성'

# 필요키워드3
need_kwd3 = '인조잔디'

# 제외키워드
except_kwd = 'CCTV|cctv|LED|led|가로등|가속기|간접등|간판|개방|검진|경관|계단|골프장|관리|광고|교량|교실|교육|교차로|균열|그늘막|그림|급수관|급식|난방기|내진|냉방기|네트워크|노상|노점|농기계|단열|데이터|도로개설|도로방음벽|도로정비|도로확장|도서관|디자인|뚜껑|막구조물|맨홀|모노레일|모니터링|미끄럼|미세먼지|미술|방송|방수|방음|방음벽|배면도로|배수|버스|법률자문|벤치|벽화|보건|보도블럭|보상|보일러|보행자도로|복지|볼라드|분묘|불법|빗물받이|사용료|사워실|살포|생육환경|석면|선홈통|소프트웨어|수도|수로|수목표찰|수영장|스크린|스프링클러|승강기|시계탑|시스템|식수대|신호등|안심벨|안전|약수터|억제제|역량|연결도로|예방|예술|오염|옹벽|요양기관|운영|울타리|위생|위험수목|유아|육교|음성안내|음악|의자|인양기|인터넷|입학|자료|자전거|저장고|저장장치|전광판|전기|전신주|전통시장|정화|정화조|조도|조명|조형물|조화기|주차장|줄눈|진로진학|진입도로|창고|창출|처우|철거|취업|치안|카메라|카약|통학로|파고라|펜스|폐기물|폐열|포충기|풀장|풍차|하수|하수관로|하수도|하천|학습|행사|화장|화장실|활성화|횡단보도|휀스|흡연'

In [21]:
budget_df_today = budget_df_today[((budget_df_today['세부사업명'].str.contains(need_kwd1) & 
                                  budget_df_today['세부사업명'].str.contains(need_kwd2)) &
                                  ~budget_df_today['세부사업명'].str.contains(except_kwd)) |
                                  budget_df_today['세부사업명'].str.contains(need_kwd3)
                                  ].reset_index(drop=True)

In [22]:
# 사업명 전처리
budget_df_today['세부사업명_re'] = budget_df_today['세부사업명'].apply(clean_text)
# budget_df_today['세부사업명_re'] = budget_df_today['세부사업명_re'].str.replace('초등학교','').str.replace('중학교','').str.replace('고등학교','')
budget_df_today['세부사업명_re'] = budget_df_today['세부사업명_re'].str.strip()

budget_df_today['세부사업명_re'] = budget_df_today['세부사업명_re'].str.split().apply(lambda x: [kw for kw in x if len(kw) > 1]) ## 글자수 1 제외
budget_df_today['세부사업명_re'] = budget_df_today['세부사업명_re'].apply((' ').join)

In [23]:
# 해당 지역 추출
filtered_budget_df_temp1 = budget_df_today[budget_df_today['자치단체명'].str.contains(('|').join(dist_nm))].reset_index(drop=True)
filtered_budget_df_temp2 = budget_df_today[budget_df_today['자치단체명'].str.contains('본청')].reset_index(drop=True)

filtered_budget_df_today = pd.concat([filtered_budget_df_temp1,filtered_budget_df_temp2],axis=0).reset_index(drop=True)

In [24]:
# 편성액 기준
filtered_budget_df_today = filtered_budget_df_today[filtered_budget_df_today['편성액']!='0'].reset_index(drop=True)

#### 지자체 종합쇼핑몰 전처리

In [25]:
# Initialize the dictionary for split keywords
shop_split_kwd = {}
for i in range(all_shop_df_fin['납품요구건명_re'].str.split().apply(len).max()):
    shop_split_kwd['kwd_' + str(i)] = all_shop_df_fin['납품요구건명_re'].str.split().str[i]

In [26]:
# 매핑데이터셋
mapping_merge_df = []

In [27]:
# 종합쇼핑몰 납품요구건명 split 키워드 순서대로 맵핑
# Initialize the list to store the results
prep_budget_df_today = filtered_budget_df_today.copy()
prep_all_shop_df_fin = all_shop_df_fin.copy()

# Iterate over each keyword column
for i in range(len(shop_split_kwd)):
    keyword_column = 'kwd_' + str(i)
    
    # Get unique non-null keywords
    shop_split_kwd_temp = shop_split_kwd[keyword_column].dropna().unique()

    prep_budget_df_today[keyword_column] = prep_budget_df_today['세부사업명_re'].apply(lambda x: ' '.join(re.findall('|'.join(shop_split_kwd_temp), x)))
    prep_budget_df_today[keyword_column] = prep_budget_df_today[keyword_column].str.strip()
    prep_budget_df_today[keyword_column] = prep_budget_df_today[keyword_column].str.split().apply(lambda x : ' '.join(sorted(set(x)))).str.strip()

    prep_all_shop_df_fin[keyword_column] = prep_all_shop_df_fin['납품요구건명_re'].apply(lambda x: ' '.join(re.findall('|'.join(shop_split_kwd_temp), x)))
    prep_all_shop_df_fin[keyword_column] = prep_all_shop_df_fin[keyword_column].str.strip()
    prep_all_shop_df_fin[keyword_column] = prep_all_shop_df_fin[keyword_column].str.split().apply(lambda x : ' '.join(sorted(set(x)))).str.strip()

    mapping_merge_df_temp = pd.merge(
            prep_budget_df_today[(prep_budget_df_today[keyword_column] != '') & ~(prep_budget_df_today[keyword_column].isnull())][['지역명','자치단체명','세부사업코드','세부사업명', '세부사업명_re', keyword_column]].reset_index(drop=True),
            prep_all_shop_df_fin[(prep_all_shop_df_fin[keyword_column] != '') & ~(prep_all_shop_df_fin[keyword_column].isnull())][['수요기관명','수요기관지역명','납품요구건명', '납품요구건명_re', keyword_column]].reset_index(drop=True),
            how='left',
            on=keyword_column
            ).drop_duplicates().reset_index(drop=True)
    
    # Append the temporary DataFrame to the results list
    mapping_merge_df.append(mapping_merge_df_temp)

In [28]:
# 종합쇼핑몰 지역 키워드 맵핑 - 세부사업명 지역 및 해당지역 추출
# Initialize the columns for district keywords
prep_budget_df_today = filtered_budget_df_today.copy()
prep_all_shop_df_fin = all_shop_df_fin.copy()

sig_pat = '|'.join(dist_nm)

prep_budget_df_today['kwd_sig'] = prep_budget_df_today['세부사업명_re'].apply(lambda x: ' '.join(re.findall(sig_pat, x)))
prep_budget_df_today['kwd_sig'] = prep_budget_df_today['kwd_sig'].str.strip()
prep_budget_df_today['kwd_sig'] = prep_budget_df_today['kwd_sig'].str.split()
prep_budget_df_today = prep_budget_df_today.explode('kwd_sig')

prep_all_shop_df_fin['kwd_sig'] = prep_all_shop_df_fin['납품요구건명_re'].apply(lambda x: ' '.join(re.findall(sig_pat, x)))
prep_all_shop_df_fin['kwd_sig'] = prep_all_shop_df_fin['kwd_sig'].str.strip()
prep_all_shop_df_fin['kwd_sig'] = prep_all_shop_df_fin['kwd_sig'].str.split()
prep_all_shop_df_fin = prep_all_shop_df_fin.explode('kwd_sig')

mapping_merge_df_sig = pd.merge(
    prep_budget_df_today[(prep_budget_df_today['kwd_sig'] != '') & ~(prep_budget_df_today['kwd_sig'].isnull())][['지역명','자치단체명','세부사업코드','세부사업명', '세부사업명_re', 'kwd_sig']].reset_index(drop=True),
    prep_all_shop_df_fin[(prep_all_shop_df_fin['kwd_sig'] != '') & ~(prep_all_shop_df_fin['kwd_sig'].isnull())][['수요기관명','수요기관지역명','납품요구건명', '납품요구건명_re', 'kwd_sig']].reset_index(drop=True),
    how='left',
    on='kwd_sig'
    ).drop_duplicates().reset_index(drop=True)

# Append the district keywords DataFrame to the results list
mapping_merge_df.append(mapping_merge_df_sig)

In [29]:
# 종합쇼핑몰 지역 키워드 맵핑 - 세부사업명 지역 및 해당지역 읍면동 추출
# Initialize the columns for district keywords
prep_budget_df_today = filtered_budget_df_today.copy()
prep_all_shop_df_fin = all_shop_df_fin.copy()

emd_pat = '|'.join(set(emd_code_list))

prep_budget_df_today['kwd_emd'] = prep_budget_df_today['세부사업명_re'].apply(lambda x: ' '.join(re.findall(emd_pat, x)))
prep_budget_df_today['kwd_emd'] = prep_budget_df_today['kwd_emd'].str.strip()
prep_budget_df_today['kwd_emd'] = prep_budget_df_today['kwd_emd'].str.split()
prep_budget_df_today = prep_budget_df_today.explode('kwd_emd')

prep_all_shop_df_fin['kwd_emd'] = prep_all_shop_df_fin['납품요구건명_re'].apply(lambda x: ' '.join(re.findall(emd_pat, x)))
prep_all_shop_df_fin['kwd_emd'] = prep_all_shop_df_fin['kwd_emd'].str.strip()
prep_all_shop_df_fin['kwd_emd'] = prep_all_shop_df_fin['kwd_emd'].str.split()
prep_all_shop_df_fin = prep_all_shop_df_fin.explode('kwd_emd')

mapping_merge_df_emd = pd.merge(
    prep_budget_df_today[(prep_budget_df_today['kwd_emd'] != '') & ~(prep_budget_df_today['kwd_emd'].isnull())][['지역명','자치단체명','세부사업코드','세부사업명', '세부사업명_re', 'kwd_emd']].reset_index(drop=True),
    prep_all_shop_df_fin[(prep_all_shop_df_fin['kwd_emd'] != '') & ~(prep_all_shop_df_fin['kwd_emd'].isnull())][['수요기관명','수요기관지역명','납품요구건명', '납품요구건명_re', 'kwd_emd']].reset_index(drop=True),
    how='left',
    on='kwd_emd'
    ).drop_duplicates().reset_index(drop=True)

# Append the district keywords DataFrame to the results list
mapping_merge_df.append(mapping_merge_df_emd)

In [30]:
# 필요키워드 앞부분 추출
need_kwd_split = list(set(chain(*all_shop_df_fin['납품요구건명_re'].apply(lambda x: re.split(need_kwd1,x)).apply(''.join).str.split())))
need_kwd_split_list = list(pd.Series(need_kwd_split)[pd.Series(need_kwd_split).apply(len) >= 2])

In [31]:
# 종합쇼핑몰 지역 키워드 맵핑 - 필요키워드 앞부분 매핑
# Initialize the columns for district keywords
prep_budget_df_today = filtered_budget_df_today.copy()
prep_all_shop_df_fin = all_shop_df_fin.copy()

kwd_split_pat = '|'.join(set(need_kwd_split_list))

prep_budget_df_today['kwd_need_split'] = prep_budget_df_today['세부사업명_re'].apply(lambda x: ' '.join(re.findall(kwd_split_pat, x)))
prep_budget_df_today['kwd_need_split'] = prep_budget_df_today['kwd_need_split'].str.strip()
prep_budget_df_today['kwd_need_split'] = prep_budget_df_today['kwd_need_split'].str.split()
prep_budget_df_today = prep_budget_df_today.explode('kwd_need_split')

prep_all_shop_df_fin['kwd_need_split'] = prep_all_shop_df_fin['납품요구건명_re'].apply(lambda x: ' '.join(re.findall(kwd_split_pat, x)))
prep_all_shop_df_fin['kwd_need_split'] = prep_all_shop_df_fin['kwd_need_split'].str.strip()
prep_all_shop_df_fin['kwd_need_split'] = prep_all_shop_df_fin['kwd_need_split'].str.split()
prep_all_shop_df_fin = prep_all_shop_df_fin.explode('kwd_need_split')

mapping_merge_df_need_split = pd.merge(
    prep_budget_df_today[(prep_budget_df_today['kwd_need_split'] != '') & ~(prep_budget_df_today['kwd_need_split'].isnull())][['지역명','자치단체명','세부사업코드','세부사업명', '세부사업명_re', 'kwd_need_split']].reset_index(drop=True),
    prep_all_shop_df_fin[(prep_all_shop_df_fin['kwd_need_split'] != '') & ~(prep_all_shop_df_fin['kwd_need_split'].isnull())][['수요기관명','수요기관지역명','납품요구건명', '납품요구건명_re', 'kwd_need_split']].reset_index(drop=True),
    how='left',
    on='kwd_need_split'
    ).drop_duplicates().reset_index(drop=True)

# Append the district keywords DataFrame to the results list
mapping_merge_df.append(mapping_merge_df_need_split)

In [32]:
# 종합쇼핑몰 지역 키워드 맵핑 - 필요키워드 매핑
# Initialize the columns for district keywords
prep_budget_df_today = filtered_budget_df_today.copy()
prep_all_shop_df_fin = all_shop_df_fin.copy()

prep_budget_df_today['kwd_need'] = prep_budget_df_today['세부사업명_re'].apply(lambda x: ' '.join(re.findall(need_kwd1, x)))
prep_budget_df_today['kwd_need'] = prep_budget_df_today['kwd_need'].str.strip()
prep_budget_df_today['kwd_need'] = prep_budget_df_today['kwd_need'].str.split()
prep_budget_df_today = prep_budget_df_today.explode('kwd_need')

prep_all_shop_df_fin['kwd_need'] = prep_all_shop_df_fin['납품요구건명_re'].apply(lambda x: ' '.join(re.findall(need_kwd1, x)))
prep_all_shop_df_fin['kwd_need'] = prep_all_shop_df_fin['kwd_need'].str.strip()
prep_all_shop_df_fin['kwd_need'] = prep_all_shop_df_fin['kwd_need'].str.split()
prep_all_shop_df_fin = prep_all_shop_df_fin.explode('kwd_need')

mapping_merge_df_need = pd.merge(
    prep_budget_df_today[(prep_budget_df_today['kwd_need'] != '') & ~(prep_budget_df_today['kwd_need'].isnull())][['지역명','자치단체명','세부사업코드','세부사업명', '세부사업명_re', 'kwd_need']].reset_index(drop=True),
    prep_all_shop_df_fin[(prep_all_shop_df_fin['kwd_need'] != '') & ~(prep_all_shop_df_fin['kwd_need'].isnull())][['수요기관명','수요기관지역명','납품요구건명', '납품요구건명_re', 'kwd_need']].reset_index(drop=True),
    how='left',
    on='kwd_need'
    ).drop_duplicates().reset_index(drop=True)

# Append the district keywords DataFrame to the results list
mapping_merge_df.append(mapping_merge_df_need)

In [33]:
# 키워드별 데이터셋 병합
mapping_merge_df_fin = pd.concat(mapping_merge_df, axis=0).drop_duplicates()
mapping_merge_df_fin = mapping_merge_df_fin[~mapping_merge_df_fin['납품요구건명'].isnull()].reset_index(drop=True)

In [34]:
# 모든 키워드 합
kwd_columns = ['kwd_0','kwd_1','kwd_2','kwd_3','kwd_4','kwd_5','kwd_6','kwd_7','kwd_8','kwd_sig','kwd_emd','kwd_need_split','kwd_need']
mapping_merge_df_fin['all_kwd'] = mapping_merge_df_fin[kwd_columns].apply(lambda x : (' ').join(x.dropna()),axis=1)
mapping_merge_df_fin['all_kwd'] = mapping_merge_df_fin['all_kwd'].str.strip()
mapping_merge_df_fin['all_kwd'] = mapping_merge_df_fin['all_kwd'].str.split().apply(lambda x : ' '.join(sorted(set(x)))).str.strip()
mapping_merge_df_fin.drop(columns=kwd_columns, inplace=True)

In [35]:
# 지역명 추출1
mapping_merge_df_fin['수요기관지역명_split1'] = mapping_merge_df_fin['수요기관지역명'].str.split().str[0]

# Define the replacements
replacements = {
    '전라북도': '전북',
    '전라남도': '전남',
    '경상북도': '경북',
    '경상남도': '경남',
    '충청북도': '충북',
    '충청남도': '충남'
}

# Apply the replacements
mapping_merge_df_fin['수요기관지역명_split1'] = mapping_merge_df_fin['수요기관지역명_split1'].replace(replacements)
mapping_merge_df_fin['수요기관지역명_split1'] = mapping_merge_df_fin['수요기관지역명_split1'].str[:2]

In [36]:
# 지역명 추출2
mapping_merge_df_fin['수요기관지역명_split2'] = mapping_merge_df_fin['수요기관지역명'].str.split().str[1]
dist_sig_pat = '|'.join(mapping_merge_df_fin['수요기관지역명_split2'].dropna().unique())

mapping_merge_df_fin['자치단체명_re'] = mapping_merge_df_fin['자치단체명'].apply(lambda x: ' '.join(re.findall(dist_sig_pat, x)))
mapping_merge_df_fin['자치단체명_re'] = mapping_merge_df_fin['자치단체명_re'].str.strip()
mapping_merge_df_fin['자치단체명_re'] = mapping_merge_df_fin['자치단체명_re'].str.split()
mapping_merge_df_fin = mapping_merge_df_fin.explode('자치단체명_re')

In [37]:
# 제외 지역 제거
filtered_mapping_merge_df_fin = mapping_merge_df_fin[mapping_merge_df_fin['자치단체명_re']!=''].drop_duplicates().reset_index(drop=True)

In [38]:
# 지역 일치 확인1
mapping_merge_df_fin_temp1 = filtered_mapping_merge_df_fin[filtered_mapping_merge_df_fin['지역명']==filtered_mapping_merge_df_fin['수요기관지역명_split1']].drop_duplicates().reset_index(drop=True)
mapping_merge_df_fin_temp1 = mapping_merge_df_fin_temp1[mapping_merge_df_fin_temp1['자치단체명'].str.contains('|'.join(mapping_merge_df_fin_temp1['수요기관지역명_split2'].dropna().unique()))].drop_duplicates().reset_index(drop=True)
mapping_merge_df_fin_temp1 = mapping_merge_df_fin_temp1[mapping_merge_df_fin_temp1['자치단체명_re']==mapping_merge_df_fin_temp1['수요기관지역명_split2']].drop_duplicates().reset_index(drop=True)

# 본청 포함 확인
mapping_merge_df_fin_temp2 = filtered_mapping_merge_df_fin[filtered_mapping_merge_df_fin['자치단체명'].str.contains('본청')].drop_duplicates().reset_index(drop=True)

# 데이터 병합
mapping_merge_df_final = pd.concat([mapping_merge_df_fin_temp1,mapping_merge_df_fin_temp2], axis=0).drop_duplicates().reset_index(drop=True)

In [39]:
# 그룹화할 컬럼 리스트
group_cols = ['지역명', '자치단체명', '세부사업코드', '세부사업명', '세부사업명_re', '수요기관명', '수요기관지역명', '납품요구건명', '납품요구건명_re','수요기관지역명_split1', '자치단체명_re', '수요기관지역명_split2']

# 그룹화 및 all_kwd 컬럼 조인
mapping_merge_df_final_grp = mapping_merge_df_final.groupby(group_cols).agg({'all_kwd': ' '.join}).reset_index()
mapping_merge_df_final_grp['all_kwd'] = mapping_merge_df_final_grp['all_kwd'].str.strip()
mapping_merge_df_final_grp['all_kwd'] = mapping_merge_df_final_grp['all_kwd'].str.split().apply(lambda x : ' '.join(sorted(set(x)))).str.strip()

In [40]:
# 필요키워드 기준
mapping_merge_df_final_grp['세부사업명_키워드1'] = mapping_merge_df_final_grp['세부사업명'].apply(lambda x: ' '.join(re.findall(need_kwd1, x)))
mapping_merge_df_final_grp['납품요구건명_키워드1'] = mapping_merge_df_final_grp['납품요구건명'].apply(lambda x: ' '.join(re.findall(need_kwd1, x)))

# 사업명 지역 추출
mapping_merge_df_final_grp['세부사업명_지역'] = mapping_merge_df_final_grp['세부사업명'].apply(lambda x: ' '.join(re.findall(emd_pat, x)))
mapping_merge_df_final_grp['납품요구건명_지역'] = mapping_merge_df_final_grp['납품요구건명'].apply(lambda x: ' '.join(re.findall(emd_pat, x)))

# 필요키워드 패턴 기준
need_kwd_pat = '\w+' + ('|\w+').join(need_kwd1.split('|'))

mapping_merge_df_final_grp['세부사업명_키워드2'] = mapping_merge_df_final_grp['세부사업명'].apply(lambda x: ' '.join(re.findall(need_kwd_pat, x)))
mapping_merge_df_final_grp['납품요구건명_키워드2'] = mapping_merge_df_final_grp['납품요구건명'].apply(lambda x: ' '.join(re.findall(need_kwd_pat, x)))

In [41]:
def check_inclusion1(row):
    set1 = set(row['세부사업명_키워드1'])
    set2 = set(row['납품요구건명_키워드1'])
    return bool(set1 & set2)

In [42]:
# 필요키워드 모두 없는것 제외
mapping_merge_df_final_grp_temp1 = mapping_merge_df_final_grp[(mapping_merge_df_final_grp['세부사업명_키워드1']!='')&(mapping_merge_df_final_grp['납품요구건명_키워드1']!='')].reset_index(drop=True)

# 필요키워드 포함여부 확인
mapping_merge_df_final_grp_temp1['세부사업명_키워드1'] = mapping_merge_df_final_grp_temp1['세부사업명_키워드1'].str.split() ## 키워드별 정리
mapping_merge_df_final_grp_temp1['납품요구건명_키워드1'] = mapping_merge_df_final_grp_temp1['납품요구건명_키워드1'].str.split() ## 키워드별 정리

mapping_merge_df_final_grp_temp1 = mapping_merge_df_final_grp_temp1[mapping_merge_df_final_grp_temp1.apply(check_inclusion1,axis=1)].reset_index(drop=True)

mapping_merge_df_final_grp_temp1['세부사업명_키워드1'] = mapping_merge_df_final_grp_temp1['세부사업명_키워드1'].apply(lambda x : ' '.join(sorted(set(x)))).str.strip()
mapping_merge_df_final_grp_temp1['납품요구건명_키워드1'] = mapping_merge_df_final_grp_temp1['납품요구건명_키워드1'].apply(lambda x : ' '.join(sorted(set(x)))).str.strip()

In [43]:
def check_inclusion_dist(row):
    set1 = set(row['세부사업명_지역'])
    set2 = set(row['납품요구건명_지역'])
    return bool(set1 & set2)

In [44]:
# 지역 없는것 제외
mapping_merge_df_final_grp_temp2 = mapping_merge_df_final_grp[(mapping_merge_df_final_grp['세부사업명_지역']!='')&(mapping_merge_df_final_grp['납품요구건명_지역']!='')].reset_index(drop=True)

# 사업명 지역 포함여부 확인
mapping_merge_df_final_grp_temp2['세부사업명_지역'] = mapping_merge_df_final_grp_temp2['세부사업명_지역'].str.split()
mapping_merge_df_final_grp_temp2['납품요구건명_지역'] = mapping_merge_df_final_grp_temp2['납품요구건명_지역'].str.split()

mapping_merge_df_final_grp_temp2 = mapping_merge_df_final_grp_temp2[mapping_merge_df_final_grp_temp2.apply(check_inclusion_dist,axis=1)].reset_index(drop=True)

mapping_merge_df_final_grp_temp2['세부사업명_지역'] = mapping_merge_df_final_grp_temp2['세부사업명_지역'].apply(lambda x : ' '.join(sorted(set(x)))).str.strip() ## 키워드별 정리
mapping_merge_df_final_grp_temp2['납품요구건명_지역'] = mapping_merge_df_final_grp_temp2['납품요구건명_지역'].apply(lambda x : ' '.join(sorted(set(x)))).str.strip() ## 키워드별 정리

In [45]:
def check_inclusion2(row):
    set1 = set(row['세부사업명_키워드2'])
    set2 = set(row['납품요구건명_키워드2'])
    return bool(set1 & set2)

In [46]:
# 필요키워드패턴 모두 없는것 제외
mapping_merge_df_final_grp_temp3 = mapping_merge_df_final_grp[(mapping_merge_df_final_grp['세부사업명_키워드2']!='')&(mapping_merge_df_final_grp['납품요구건명_키워드2']!='')].reset_index(drop=True)

# 필요키워드 패턴 포함여부 확인
mapping_merge_df_final_grp_temp3['세부사업명_키워드2'] = mapping_merge_df_final_grp_temp3['세부사업명_키워드2'].str.split() ## 키워드별 정리
mapping_merge_df_final_grp_temp3['납품요구건명_키워드2'] = mapping_merge_df_final_grp_temp3['납품요구건명_키워드2'].str.split() ## 키워드별 정리

mapping_merge_df_final_grp_temp3 = mapping_merge_df_final_grp_temp3[mapping_merge_df_final_grp_temp3.apply(check_inclusion2,axis=1)].reset_index(drop=True)

mapping_merge_df_final_grp_temp3['세부사업명_키워드2'] = mapping_merge_df_final_grp_temp3['세부사업명_키워드2'].apply(lambda x : ' '.join(sorted(set(x)))).str.strip()
mapping_merge_df_final_grp_temp3['납품요구건명_키워드2'] = mapping_merge_df_final_grp_temp3['납품요구건명_키워드2'].apply(lambda x : ' '.join(sorted(set(x)))).str.strip()

In [47]:
# 필터링 데이터셋 병합
mapping_merge_df_final_grp_fin = pd.concat([mapping_merge_df_final_grp_temp1,mapping_merge_df_final_grp_temp2,mapping_merge_df_final_grp_temp3]).drop_duplicates().reset_index(drop=True)

In [48]:
# 모든 키워드 합 중복 제거
kwd_columns = ['all_kwd','세부사업명_키워드1', '납품요구건명_키워드1', '세부사업명_지역', '납품요구건명_지역', '세부사업명_키워드2', '납품요구건명_키워드2']
mapping_merge_df_final_grp_fin['all_kwd_fin'] = mapping_merge_df_final_grp_fin[kwd_columns].apply(lambda x : (' ').join(x.dropna()),axis=1)
mapping_merge_df_final_grp_fin['all_kwd_fin'] = mapping_merge_df_final_grp_fin['all_kwd_fin'].str.strip()
mapping_merge_df_final_grp_fin['all_kwd_fin'] = mapping_merge_df_final_grp_fin['all_kwd_fin'].str.split().apply(lambda x : ' '.join(sorted(set(x)))).str.strip()
# mapping_merge_df_final_grp_fin.drop(columns=kwd_columns, inplace
# =True)

mapping_merge_df_final_grp_fin = mapping_merge_df_final_grp_fin.drop_duplicates(['지역명', '자치단체명', '세부사업코드', '세부사업명', '세부사업명_re', '수요기관명', '수요기관지역명',
                                                                                 '납품요구건명', '납품요구건명_re', '수요기관지역명_split1', '자치단체명_re', '수요기관지역명_split2', 'all_kwd_fin']).reset_index(drop=True)

In [49]:
# 최종필터링1 - 지역 일치
mapping_merge_df_final_grp_fin_temp1 = mapping_merge_df_final_grp_fin[(mapping_merge_df_final_grp_fin['세부사업명_지역']!='') & (mapping_merge_df_final_grp_fin['납품요구건명_지역']!='')]
mapping_merge_df_final_grp_fin_temp1 = mapping_merge_df_final_grp_fin_temp1[mapping_merge_df_final_grp_fin_temp1['세부사업명_지역']==mapping_merge_df_final_grp_fin_temp1['납품요구건명_지역']]

In [50]:
# 최종필터링2 - 필수키워드 패턴 포함 & 지역 불일치 제거
mapping_merge_df_final_grp_fin_temp2 = mapping_merge_df_final_grp_fin[(mapping_merge_df_final_grp_fin['세부사업명_키워드2']!='')&(mapping_merge_df_final_grp_fin['납품요구건명_키워드2']!='')].reset_index(drop=True)

# 필요키워드 패턴 포함여부 확인
mapping_merge_df_final_grp_fin_temp2['세부사업명_키워드2'] = mapping_merge_df_final_grp_fin_temp2['세부사업명_키워드2'].str.split() ## 키워드별 정리
mapping_merge_df_final_grp_fin_temp2['납품요구건명_키워드2'] = mapping_merge_df_final_grp_fin_temp2['납품요구건명_키워드2'].str.split() ## 키워드별 정리

mapping_merge_df_final_grp_fin_temp2 = mapping_merge_df_final_grp_fin_temp2[mapping_merge_df_final_grp_fin_temp2.apply(check_inclusion2,axis=1)].reset_index(drop=True)

mapping_merge_df_final_grp_fin_temp2['세부사업명_키워드2'] = mapping_merge_df_final_grp_fin_temp2['세부사업명_키워드2'].apply(lambda x : ' '.join(sorted(set(x)))).str.strip()
mapping_merge_df_final_grp_fin_temp2['납품요구건명_키워드2'] = mapping_merge_df_final_grp_fin_temp2['납품요구건명_키워드2'].apply(lambda x : ' '.join(sorted(set(x)))).str.strip()

# 지역 불일치 제거
mapping_merge_df_final_grp_fin_temp2 = mapping_merge_df_final_grp_fin_temp2[mapping_merge_df_final_grp_fin_temp2['세부사업명_지역']==mapping_merge_df_final_grp_fin_temp2['납품요구건명_지역']]

In [51]:
# 최종 필터링 데이터셋
mapping_merge_df_final_grp_final = pd.concat([mapping_merge_df_final_grp_fin_temp1,mapping_merge_df_final_grp_fin_temp2]).drop_duplicates().reset_index(drop=True)
mapping_merge_df_final_grp_final.drop(columns = 'all_kwd', inplace=True)

In [52]:
# # 내보내기
# mapping_final_col = ['지역명', '자치단체명', '세부사업명', '수요기관명', '수요기관지역명','납품요구건명', 'all_kwd_fin']
# mapping_merge_df_final_grp_final = mapping_merge_df_final_grp_final[mapping_final_col]
# mapping_merge_df_final_grp_final.rename(columns={'all_kwd_fin':'키워드'},inplace=True)
# mapping_merge_df_final_grp_final.to_csv('C:/py_src/midoproject/result/filtered_mapping_budget_final.csv',encoding='utf-8-sig',index=False)

In [53]:
mapping_merge_df_final_grp_final.to_csv('C:/py_src/midoproject/result/filtered_mapping_budget_df.csv', encoding='utf-8-sig', index=False)

In [54]:
# ## 이미 진행사업 확인 제거
# except_mapping_list = []
except_mapping_list = ['정림리 테니스장 시설개선사업','소초면 흥양리 체육시설 조성(전환사업)','호저면 만종리 다목적구장 조성(전환사업)','남면 생활체육공원 운동장 개보수공사[자체]',
                       '화촌면 생활체육공원 축구장 인조잔디 교체(전환사업)','북방면 공설운동장 규격확장 및 인조잔디 교체사업(전환사업)','상면생활체육공원 시설 개보수','주교동 고가하부 체육시설 환경개선공사',
                       '남한산성면 상번천리 체육시설 조성','중면 축구장 개선사업(도비)','원삼면 공공체육시설 설치','용동중학교 운동장 조성','하갈동 체육시설 개선공사[특별조정교부금사업]',
                       '마장면 체육공원 인조잔디 교체','신둔면 체육공원 풋살장 및 족구장 인조잔디 설치(보조)','일동면 기산리 풋살장 조성사업','진영공설운동장 시설물 개선공사(전환사업)',
                       '동면 상동체육공원 정비공사','고령우곡면게이트볼장시설물보수공사','석적리틀야구장 정비사업','용동공원 축구장 시설개선사업(전환사업)','화원 본리리 족구장 조성',
                       '신안공설운동장족구장조성(전환사업)','미산면 다목적 잔디구장 조성','내수읍(덕암천) 족구장 정비사업','교1동 풋살장 인조잔디 교체 공사','간성족구장 조성(전환사업)',
                       '용하체육공원 풋살장 환경개선사업','상남테니스장 조성사업[자체]','서화 전천후게이트볼장 개보수 공사[기금]','둔내종합체육공원 축구장 인조잔디 교체공사(도)(전환)',
                       '둔내종합체육공원 축구장 인조잔디 교체공사(도)(전환)','청평테니스장 시설 개보수','설악야구장 시설 개보수','설악야구장 시설 개보수','설악생활체육공원 시설 개보수',
                       '쌈지공원 조성(도비)(전환사업)','철산게이트볼장 환경개선공사','한내근린공원 공원시설 정비사업','송정남근린공원 조성사업','구향공원 다목적구장 설치',
                       '남양주체육문화센터 테니스장 노후시설 개보수','성무정공원 노후시설 환경개선사업','성남종합운동장 실내체육관 지붕 개보수','만석공원 전천후 테니스장 조성',
                       '대부둑공원 내 체육시설 조성(도비)','당산어린이공원 다목적구장 정비','만석공원 게이트볼장 정비','동부지역 생활체육공원 정비공사','양주시 실내족구장 조성사업',
                       '새릉골어린이공원 어린이놀이시설 정비사업','남사게이트볼장 실내환경 개선공사','내손체육공원 축구장 정비',"청석게이트볼장 인조잔디 교체공사('23년 특조)",'청북 다목적체육관 조성',
                       '국제대학교 운동장 인조잔디 설치 공사(도비)(전환사업)','영북테니스장 시설개선 사업(전환사업)','모충공원조성','장승배기 생태공원 체육시설 정비사업(전환사업)','삼정자공원 조성',
                       '우곡문화공원 조성','[지특]필드하키장 조성','청도천체육시설(축구장)정비(전환)','포항환여풋살장시설보수<보조>','율하체육공원 테니스장 확장공사','진밭골 생활체육시설 조성',
                       '신호공원 인조잔디운동장 노후 인조잔디 교체','백운포체육공원 인조2축구장 시설개선','덕포공원 게이트볼장 정비사업','해누리체육공원 노후시설 개보수','남산레포츠공원 체육시설 개선',
                       '남산레포츠공원 체육시설 개선','언양게이트볼장 조성사업','사염공원, 까투렴공원, 물병자리공원 내 노후시설물 정비사업','압해게이트볼장인조잔디조성(전환사업)',
                       '영암종합운동장 풋살장조성(전환사업)','상무평화공원 축구장 개보수(전환사업)','지천생태공원 종합운동장 조성(도/전환)(전환사업)','완주군 과학산업단지 어린이체육관 조성사업(전환사업)',
                       '이서 지사울공원 축구장 인조잔디 교체사업(전환사업)','이서 지사울공원 축구장 인조잔디 교체사업(전환사업)','덕진체련공원 족구장 인조잔디 교체','공주 시립테니스장 개선',
                       '시민운동장 내부시설 개보수','부여종합운동장 테니스코트 돔 조성','축구종합센터 생활체육 부대시설 설치사업','축구종합센터 생활체육 부대시설 설치사업','천안초등학교 인근 공원조성',
                       '체육진흥시설 지원(괴산스포츠타운 조성)','체육진흥시설 지원(괴산스포츠타운 조성)','감곡생활체육공원 정비공사','옥산생활체육공원 인조잔디 교체공사(특별조정교부금)',
                       '옥산생활체육공원 다목적체육관 시설개선사업','성무생활체육시설 개선사업']

In [55]:
# 필요컬럼추출
need_col = ['지역명', '자치단체명', '세부사업명', '분야명', '부문명', '예산현액', '국비', '시도비', '시군구비', '기타', '지출액', '편성액']

# filtered_budget_df = budget_sheet_df[~budget_sheet_df['세부사업명'].isin(except_mapping_list)].reset_index(drop=True)
filtered_budget_df = budget_df_today[~budget_df_today['세부사업명'].isin(except_mapping_list)][need_col]
filtered_budget_df = filtered_budget_df[filtered_budget_df['편성액']!='0'].sort_values(['지역명']).reset_index(drop=True) ## 편성액 0이면 끝난 사업으로 판단

# 자치단체명 형식
filtered_budget_df['자치단체명'] = np.where(filtered_budget_df['자치단체명'].str.contains('본청'),filtered_budget_df['자치단체명'],filtered_budget_df['자치단체명'].str[2:])
filtered_budget_df

,지역명,자치단체명,세부사업명,분야명,부문명,예산현액,국비,시도비,시군구비,기타,지출액,편성액
0,강원,고성군,거진테니스장 비가림시설 설치(전환사업),문화및관광,체육,4700000000,0,3055000000,1645000000,0,0,1295000000
1,강원,삼척시,학교체육(엘리트) 육성,교육,유아및초중등교육,546400000,0,0,546400000,0,452100000,546400000
2,강원,삼척시,생활문화센터(생활문화 체육공원) 조성(국고),문화및관광,문화예술,1884000000,283000000,0,1601000000,0,524807800,1115000000
3,강원,삼척시,제60회 강원특별자치도민 체육대회 경기장 유지보수,문화및관광,체육,1800000000,0,0,1800000000,0,0,1800000000
4,강원,삼척시,생활문화체육공원 조성(도)(전환사업),문화및관광,체육,11188252600,315702320,4953060030,5919490250,0,1950362560,4602000000
...,...,...,...,...,...,...,...,...,...,...,...,...
2401,충북,단양군,단양생태체육공원 정비사업,국토및지역개발,수자원,480403000,0,0,480403000,0,390694190,80000000
2402,충북,단양군,공원시설물 유지보수,국토및지역개발,지역및도시,392456510,0,0,392456510,0,117046000,300000000
2403,충북,증평군,체육시설 정비,문화및관광,체육,148600000,0,0,148600000,0,126173780,148600000
2404,충북,음성군,체육시설 확충 및 보수,문화및관광,체육,983058670,0,0,983058670,0,673885910,882268000


In [56]:
# Define the replacements
replacements = {
    '강원': '강원특별자치도', 
    '경기': '경기도', 
    '경남': '경상남도',
    '경북': '경상북도', 
    '광주': '광주광역시', 
    '대구': '대구광역시', 
    '대전': '대전광역시', 
    '부산': '부산광역시', 
    '서울': '서울특별시',
    '세종': '세종특별자치시', 
    '울산': '울산광역시', 
    '인천': '인천광역시', 
    '전남': '전라남도',
    '전북': '전북특별자치도', 
    '제주': '제주특별자치도', 
    '충남':'충청남도', 
    '충북':'충청북도'
}

# Apply the replacements
budget_df_today['지역명'] = budget_df_today['지역명'].replace(replacements)
filtered_budget_df['지역명'] = filtered_budget_df['지역명'].replace(replacements)
filtered_budget_df['자치단체명'] = np.where(filtered_budget_df['자치단체명']=='부산진구', '진구', filtered_budget_df['자치단체명'])

filtered_budget_df = filtered_budget_df.sort_values(['지역명','자치단체명']).reset_index(drop=True)

In [57]:
# 지자체 예산서 링크
budget_link_df = get_dataframe_from_bigquery('RAW_DATA','budget_link',midopluskey_path)
budget_link_df = budget_link_df.drop('collection_Date',axis=1)
budget_link_df['자치단체명'] = np.where(budget_link_df['자치단체명']=='본청',budget_link_df['지역명'] + budget_link_df['자치단체명'], budget_link_df['자치단체명'])

budget_link_df['지역명'] = budget_link_df['지역명'].replace(replacements)

In [58]:
# 지자체 예산서, 예산서 링크 병합
filtered_budget_df = pd.merge(filtered_budget_df, budget_link_df, how='left', on=['지역명','자치단체명'])
filtered_budget_df

,지역명,자치단체명,세부사업명,분야명,부문명,예산현액,국비,시도비,시군구비,기타,지출액,편성액,URL
0,강원특별자치도,강릉시,강릉 통일공원 「하늘숲전망대·스카이워크」조성(전환사업),문화및관광,관광,2850000000,0,1850000000,1000000000,0,2850000000,2850000000,https://www.gn.go.kr/www/contents.do?key=6332
1,강원특별자치도,강릉시,솟대공원 정비사업,문화및관광,관광,30000000,0,0,30000000,0,0,30000000,https://www.gn.go.kr/www/contents.do?key=6332
2,강원특별자치도,강릉시,생활체육개선 지원사업,문화및관광,체육,17690000,0,5307000,12383000,0,17690000,17690000,https://www.gn.go.kr/www/contents.do?key=6332
3,강원특별자치도,강릉시,학교체육 육성 지원,문화및관광,체육,430000000,0,0,430000000,0,420000000,430000000,https://www.gn.go.kr/www/contents.do?key=6332
4,강원특별자치도,강릉시,체육시설 정비,문화및관광,체육,708560000,0,0,708560000,0,442132470,682600000,https://www.gn.go.kr/www/contents.do?key=6332
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2401,충청북도,충주시,어린이 체육공원 조성,문화및관광,체육,4681243410,1487359310,1050000000,2143884100,0,1356130500,1700000000,https://www.chungju.go.kr/www/selectBbsNttList...
2402,충청북도,충주시,전천후 론볼장 등 체육시설 조성(전환사업),문화및관광,체육,4664586550,0,2015000000,2649586550,0,4576525050,3500000000,https://www.chungju.go.kr/www/selectBbsNttList...
2403,충청북도,충주시,북부생활체육공원 조성,문화및관광,체육,29971000,0,0,29971000,0,16971000,13000000,https://www.chungju.go.kr/www/selectBbsNttList...
2404,충청북도,충주시,탄금공원 3단계 2구역 조성,문화및관광,관광,130000000,0,0,130000000,0,0,100000000,https://www.chungju.go.kr/www/selectBbsNttList...


In [59]:
# # 스프레드시트 ID (URL에서 확인 가능)
# business_sheet_id = '166xdkZYI-SDNwdEiI6-Kt-p1wsTwFTAYAVfuCiFly0E' ## 미도플러스 사업현황

# # 기존 스프레드시트 및 워크시트 열기
# WORKSHEET_NAME = '지자체'
# spreadsheet_business = gc.open_by_key(business_sheet_id)
# worksheet_business = spreadsheet_business.worksheet(WORKSHEET_NAME)

# # 기존 데이터 지우기
# worksheet_business.clear()

# # 데이터 프레임을 스프레드 시트에 덮어쓰기
# worksheet_business.update([filtered_budget_df.columns.values.tolist()] + filtered_budget_df.values.tolist())

In [59]:
save_dataframe_to_bigquery(filtered_budget_df,'budget','filtered_budget_df',bigquerykey_path) ## 개인계정

Data inserted into table filtered_budget_df successfully.


In [60]:
save_dataframe_to_bigquery(filtered_budget_df,'DATA_MARTS','list_up_budget_data_final',midopluskey_path) ## 회사계정

Data inserted into table list_up_budget_data_final successfully.


#### 추경 예산서

In [125]:
# 추경 및 전환사업
budget_add_df = filtered_budget_df[filtered_budget_df['지출액']=='0'].reset_index(drop=True)

# 편성액 적은 사업 제거
budget_add_df = budget_add_df[budget_add_df['편성액'].astype(float) >= 10000000].reset_index(drop=True)

# 인조잔디 관련 부문
budget_add_df = budget_add_df[budget_add_df['부문명'].isin(['지역및도시', '체육', '임업ㆍ산촌', '관광', '농업ㆍ농촌', '문화예술', '수자원'])].reset_index(drop=True)

In [127]:
# 추경예산비율 확인
budget_add_ratio_df = pd.merge(budget_add_df.groupby(['지역명','자치단체명'])['세부사업명'].count().reset_index(), 
                               filtered_budget_df.groupby(['지역명','자치단체명'])['세부사업명'].count().reset_index(),
                               how='left',on=['지역명','자치단체명'])
budget_add_ratio_df['추경비율'] = round(budget_add_ratio_df['세부사업명_x'] / budget_add_ratio_df['세부사업명_y'] * 100, 2)

In [129]:
# 추경 최종 데이터셋
budget_add_df_final = pd.merge(budget_add_df, budget_add_ratio_df[['지역명','자치단체명','추경비율']],how='left',on=['지역명','자치단체명'])
budget_add_df_final = budget_add_df_final[['지역명', '자치단체명', '세부사업명', '분야명', '부문명', '예산현액', '국비', '시도비', '시군구비', '기타', '지출액', '편성액', '추경비율', 'URL']]
budget_add_df_final = budget_add_df_final.sort_values(['지역명','자치단체명']).reset_index(drop=True)

In [130]:
# 예산 구분 확인 컬럼
budget_add_df_final['확인'] = '지방재정'

In [132]:
save_dataframe_to_bigquery(budget_add_df_final,'budget','budget_add_df_final',bigquerykey_path) ## 개인계정

Data inserted into table budget_add_df_final successfully.


In [133]:
save_dataframe_to_bigquery(budget_add_df_final,'DATA_MARTS','budget_add_df_final',midopluskey_path) ## 회사계정

Data inserted into table budget_add_df_final successfully.


In [255]:
# budget_add_df_fin.to_csv('C:\py_src\midoproject\data/지자체예산서_추경.csv', encoding='utf-8-sig',index=False)

#### 교육청 예산서

In [ ]:
# # 빅쿼리에서 불러오기(개인계정)
# edu_budget_df = get_dataframe_from_bigquery('edu','edu_budget_listup',bigquerykey_path)
# edu_budget_df.sort_values('시도').reset_index(drop=True)

In [134]:
# 빅쿼리에서 불러오기(회사계정)
edu_budget_df = get_dataframe_from_bigquery('DATA_MARTS','list_up_edu_budget_data',midopluskey_path)
edu_budget_df = edu_budget_df.rename(columns={'도광역시':'시도'})

In [135]:
# 전처리
edu_budget_df['과업명_re'] = edu_budget_df['과업명'].apply(clean_text)
# edu_budget_df['과업명_re'] = edu_budget_df['과업명_re'].str.replace('초등학교','').str.replace('중학교','').str.replace('고등학교','')
edu_budget_df['과업명_re'] = edu_budget_df['과업명_re'].str.strip()

In [136]:
# 지역명, 교육청명 종합쇼핑몰 형식과 통일
# Define the replacements
replacements = {
    '전라북도': '전북',
    '전라남도': '전남',
    '경상북도': '경북',
    '경상남도': '경남',
    '충청북도': '충북',
    '충청남도': '충남'
}

# Apply the replacements
edu_budget_df['시도_re'] = edu_budget_df['시도'].replace(replacements)
edu_budget_df['시도_re'] = edu_budget_df['시도_re'].str[:2]

edu_budget_df['시군구_re'] = np.where(edu_budget_df['시군구'].str.contains('교육지원청'),
                                   edu_budget_df['시도'] + edu_budget_df['시군구'].str.replace('교육지원청',''), edu_budget_df['시군구']) 

In [137]:
# 학교 관련 키워드 및 데이터 추출
edu_budget_df['교육청_키워드'] = edu_budget_df['과업명'].apply(lambda x: ''.join(re.findall('\\w+초|\\w+초등학교|\\w+중|\\w+중학교|\\w+고|\\w+고등학교|\\w+학교', x)))

edu_budget_df['교육청_키워드2'] = edu_budget_df['교육청_키워드'].str.split('초|중|고|학교').str[0] ## 학교명 추출

# 일반고 외 학교명 전처리
replacements = {
    '여자': '여',
    '공업': '공',
    '상업': '상',
    '농업': '농',
    '과학': '과'
}
edu_budget_df['교육청_키워드2'] = edu_budget_df['교육청_키워드2'].replace(replacements, regex=True)

In [138]:
# 학교명 등 필요키워드 추출
edu_kwd_temp = edu_budget_df['교육청_키워드2'].dropna().unique()
edu_kwd = edu_kwd_temp[edu_kwd_temp!='']

In [139]:
# 해당 필요키워드 추출
edu_budget_df['키워드'] = edu_budget_df['교육청_키워드2'].apply(lambda x: ''.join(re.findall('|'.join(edu_kwd), x))) ## 학교명
edu_budget_df['키워드2'] = edu_budget_df['교육청_키워드'].apply(lambda x: ''.join(re.findall('초|중|고', x)))

#### 교육청 종합쇼핑몰 전처리

In [140]:
all_shop_df_fin['키워드'] = all_shop_df_fin['납품요구건명'].apply(lambda x: ''.join(re.findall('|'.join(edu_kwd), x)))
all_shop_df_fin['키워드2'] = all_shop_df_fin['납품요구건명'].apply(lambda x: ''.join(re.findall('초|중|고', x)))

In [141]:
# 지역명 추출1
all_shop_df_fin['수요기관지역명_split1'] = all_shop_df_fin['수요기관지역명'].str.split().str[0]

# Define the replacements
replacements = {
    '전라북도': '전북',
    '전라남도': '전남',
    '경상북도': '경북',
    '경상남도': '경남',
    '충청북도': '충북',
    '충청남도': '충남'
}

# Apply the replacements
all_shop_df_fin['수요기관지역명_split1'] = all_shop_df_fin['수요기관지역명_split1'].replace(replacements)
all_shop_df_fin['수요기관지역명_split1'] = all_shop_df_fin['수요기관지역명_split1'].str[:2]

In [142]:
# 지역명 추출2
all_shop_df_fin['수요기관지역명_split2'] = all_shop_df_fin['수요기관지역명'].str.split().str[1]

all_shop_df_fin['수요기관지역명_split2'] = np.where(all_shop_df_fin['수요기관지역명_split2'].isnull(),
                                                  all_shop_df_fin['수요기관지역명_split1'],
                                                  all_shop_df_fin['수요기관지역명_split2'])

all_shop_df_fin['수요기관지역명_split2'] = all_shop_df_fin['수요기관지역명_split2'].fillna('')

all_shop_df_fin['수요기관지역명_split2'] = np.where(all_shop_df_fin['수요기관지역명_split2'].apply(len)<=2,
                                                  all_shop_df_fin['수요기관지역명_split2'],
                                                  all_shop_df_fin['수요기관지역명_split2'].str[:-1])

dist_sig_pat = '|'.join(all_shop_df_fin['수요기관지역명_split2'].dropna().unique())

In [143]:
# 교육청 지역명 추출
edu_budget_df['시군구_re2'] = edu_budget_df['시군구_re'].apply(lambda x: ' '.join(re.findall(dist_sig_pat, x)))
edu_budget_df['시군구_re2'] = edu_budget_df['시군구_re2'].str.strip()

In [144]:
# 학교 키워드 기준 병합
mapping_merge_edu_df_fin = pd.merge(edu_budget_df[['시도', '시군구', '과업명', '시도_re', '시군구_re', '시군구_re2', '키워드','키워드2']],
                                all_shop_df_fin[['수요기관명', '납품요구건명', '업체명', '수요기관지역명', '납품요구지청명', '수요기관지역명_split1', '수요기관지역명_split2', '키워드','키워드2']],
                                how='left', on=['키워드','키워드2'])
mapping_merge_edu_df_fin = mapping_merge_edu_df_fin[mapping_merge_edu_df_fin['키워드']!=''].drop_duplicates().reset_index(drop=True)
mapping_merge_edu_df_fin = mapping_merge_edu_df_fin[mapping_merge_edu_df_fin['납품요구건명'].notnull()].drop_duplicates().reset_index(drop=True)

In [145]:
# 지역 일치 확인
mapping_merge_edu_df_final = mapping_merge_edu_df_fin[mapping_merge_edu_df_fin['시도_re']==mapping_merge_edu_df_fin['수요기관지역명_split1']].drop_duplicates().reset_index(drop=True)
mapping_merge_edu_df_final.to_csv('C:/py_src/midoproject/result/filtered_mapping_edu_df.csv',encoding='utf-8-sig',index=False)
mapping_merge_edu_df_final

,시도,시군구,과업명,시도_re,시군구_re,시군구_re2,키워드,키워드2,수요기관명,납품요구건명,업체명,수요기관지역명,납품요구지청명,수요기관지역명_split1,수요기관지역명_split2
0,강원특별자치도,강원특별자치도교육청,봉의고등학교 인조잔디 운동장 및 우레탄조성 테니스장 조성,강원,강원특별자치도교육청,,봉의,고,강원특별자치도교육청 강원특별자치도춘천교육지원청,봉의고 인조잔디운동장 및 테니스장 조성공사 관급자재(인조잔디) 구입,지앤지텍 주식회사,강원도 춘천시,강원지방조달청,강원,춘천
1,강원특별자치도,춘천교육지원청,부안초등학교 인조잔디 운동장 및 우레탄조성,강원,강원특별자치도춘천,춘천,부안,초,강원특별자치도교육청 강원특별자치도춘천교육지원청,부안초 인조잔디운동장 조성공사 관급자재(인조잔디) 구입,(주)대건씨앤엘,강원도 춘천시,강원지방조달청,강원,춘천
2,강원특별자치도,춘천교육지원청,춘천초등학교 다목적구장조성,강원,강원특별자치도춘천,춘천,춘천,초,강원특별자치도교육청 강원특별자치도춘천교육지원청,춘천초 다목적구장 조성공사 관급자재(인조잔디) 구입,코니터프주식회사,강원도 춘천시,강원지방조달청,강원,춘천
3,강원특별자치도,춘천교육지원청,춘천초등학교 다목적구장조성,강원,강원특별자치도춘천,춘천,춘천,초,강원특별자치도교육청 강원특별자치도춘천교육지원청,춘천초 다목적구장 조성공사 관급자재(인조잔디) 구입,한국체육조경주식회사,강원도 춘천시,강원지방조달청,강원,춘천
4,경기도,경기도교육청,향동고등학교 친환경 운동장 조성 사업,경기,경기도교육청,,향동,고,경기도교육청 향동고등학교,[친환경]향동고 친환경운동장 조성을 위한 인조잔디 구입,주식회사 액션필드,경기도 고양시 덕양구,서울지방조달청,경기,고양
5,경기도,경기도교육청,광명북고등학교 친환경 운동장 조성 사업,경기,경기도교육청,,광명북,고,경기도교육청 광명북고등학교,광명북고 친환경 운동장 환경개선공사 인조잔디 조달 구입,넥스포텍 주식회사,경기도 광명시,인천지방조달청,경기,광명
6,경기도,광주하남교육지원청,매양중학교 친환경 운동장 조성 사업,경기,경기도광주하남,광주 하남,매양,중,경기도교육청 경기도광주하남교육지원청 매양중학교,매양중 인조잔디 운동장 조성에 따른 인조잔디 구입,주식회사 이원,경기도 광주시,서울지방조달청,경기,광주
7,경기도,구리남양주교육지원청,부양초등학교 친환경 운동장 조성 사업,경기,경기도구리남양주,구리 남양주,부양,초,경기도교육청 경기도구리남양주교육지원청 부양초등학교,2024년 부양초 인조잔디 재조성 사업관련 인조잔디 구매,주식회사 액션필드,경기도 구리시,서울지방조달청,경기,구리
8,경기도,김포교육지원청,신양중학교 친환경 운동장 조성 사업,경기,경기도김포,김포,신양,중,경기도교육청 경기도김포교육청 신양중학교,2024년 신양중 인조잔디 운동장 조성물품 구입(인조잔디),주식회사 이원,경기도 김포시,인천지방조달청,경기,김포
9,경기도,동두천양주교육지원청,보산초등학교 친환경 운동장 조성 사업,경기,경기도동두천양주,동두천 양주,보산,초,경기도동두천교육청 보산초등학교,보산초 친환경 운동장 조성 인조잔디 구입,주식회사 플랜에이,경기도 동두천시,서울지방조달청,경기,동두천


In [146]:
# ## 이미 진행사업 확인 제거
# except_mapping_list = []
except_mapping_list_edu = ['봉의고등학교 인조잔디 운동장 및 우레탄조성 테니스장 조성','부안초등학교 인조잔디 운동장 및 우레탄조성','춘천초등학교 다목적구장조성','춘천초등학교 다목적구장조성',
                           '향동고등학교 친환경 운동장 조성 사업','광명북고등학교 친환경 운동장 조성 사업','매양중학교 친환경 운동장 조성 사업','부양초등학교 친환경 운동장 조성 사업',
                           '신양중학교 친환경 운동장 조성 사업','보산초등학교 친환경 운동장 조성 사업','백학중학교 친환경 운동장 조성 사업','장호원초등학교 친환경 운동장 조성 사업',
                           '광주석산고등학교(인조잔디) (학교회계전출금)','건국중학교 인조잔디 (재) 조성','면천중학교 운동장 교체','창원상남초등학교 인조잔디 교체','대구공업고등학교 축구장 인조잔디 재조성',
                           '대륜고등학교 축구장 인조잔디 재조성','상남중학교 개방형 운동장 조성','상대초등학교(다목적구장 재조성)','상대초등학교(운동장 재조성)','석교중학교 운동장 보수',
                           '익산고등학교 인조잔디 조성','강경여자중학교 인조잔디 교체']

In [147]:
# 최종 교육청 데이터셋
edu_budget_df_final = edu_budget_df[~edu_budget_df['과업명'].isin(except_mapping_list_edu)].reset_index(drop=True)
edu_budget_df_final = edu_budget_df_final[['시도', '시군구', '구분', '과업명', '삭제', '금액', '면적', '예산집행']].sort_values('시도').reset_index(drop=True)

In [148]:
# # 스프레드시트 ID (URL에서 확인 가능)
# business_sheet_id = '166xdkZYI-SDNwdEiI6-Kt-p1wsTwFTAYAVfuCiFly0E' ## 미도플러스 사업현황

# # 기존 스프레드시트 및 워크시트 열기
# WORKSHEET_NAME = '교육청'
# spreadsheet_edu = gc.open_by_key(business_sheet_id)
# worksheet_edu = spreadsheet_edu.worksheet(WORKSHEET_NAME)

# # 기존 데이터 지우기
# worksheet_edu.clear()

# # 데이터 프레임을 스프레드 시트에 덮어쓰기
# worksheet_edu.update([edu_budget_df_final.columns.values.tolist()] + edu_budget_df_final.values.tolist())

In [149]:
save_dataframe_to_bigquery(edu_budget_df_final,'edu','edu_budget_listup',bigquerykey_path) ## 개인계정

Data inserted into table edu_budget_listup successfully.


In [150]:
save_dataframe_to_bigquery(edu_budget_df_final,'DATA_MARTS','listup_edu_budget_data_final',midopluskey_path) ## 회사계정

Data inserted into table listup_edu_budget_data_final successfully.
